In [ ]:
# Get the dependancies
# !apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
# Import dependancies
import findspark
findspark.init()

import pyspark, re
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize session
spark = SparkSession.builder.getOrCreate()

# Classification code

In [ ]:
# Download and extract the data
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Magazine_Subscriptions.json.gz --no-check-certificate
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/All_Beauty.json.gz --no-check-certificate
!gzip -d Magazine_Subscriptions.json.gz

--2023-05-17 00:47:14--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Magazine_Subscriptions.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 12875273 (12M) [application/x-gzip]
Saving to: ‘Magazine_Subscriptions.json.gz’

Magazine_Subscripti 100%[===================>]  12.28M  40.1MB/s    in 0.3s    

2023-05-17 00:47:15 (40.1 MB/s) - ‘Magazine_Subscriptions.json.gz’ saved [12875273/12875273]



In [ ]:
# View the data format
!head -n 10 Magazine_Subscriptions.json

{"overall": 5.0, "vote": "9", "verified": false, "reviewTime": "11 8, 2001", "reviewerID": "AH2IFH762VY5U", "asin": "B00005N7P0", "reviewerName": "ted sedlmayr", "reviewText": "for computer enthusiast, MaxPC is a welcome sight in your mailbox. i can remember for years savorying every page of \"boot\" (as it was called in beginning) as i was (and still am) obcessed with PC's. Anyone, from advanced users - to beginners looking for knowledge - can profit from every issue of MaxPC. the icing on the cake is the subscription that comes with a CD-ROM as it is packed with demos, utilities, and other useful apps (very helpful for those not blessed with broadband connections). Until I discovered the community of hardware enthusiast web sites, MaxPC, formerly \"boot\", was my only really informative source for computing news and articles. To this day, i consider my subscription to it worth more than 10 subscriptions to most other computing mags. I can't wait until they merge with DVD media and ma

In [ ]:
# Define a function to apply the pattern to each summary
def clean_summary(summary):
    if summary is None:
        return None
    else:
        return pattern.sub(' ', summary)

In [ ]:
# Data Preprocessing
magazine_subscription_dataset = spark.read.format("json").option("header","true").load("Magazine_Subscriptions.json")
magazine_subscription_dataset = magazine_subscription_dataset.select('overall', 'reviewerID', 'summary', 'unixReviewTime', 'reviewText')

# remove null values for any column
magazine_subscription_dataset = magazine_subscription_dataset.filter(magazine_subscription_dataset.reviewText.isNotNull())

# make primary key with col 1 and 3
magSub_df = magazine_subscription_dataset
magSub_df = magSub_df.select(concat(magSub_df.reviewerID,magSub_df.unixReviewTime)
              .alias("primary_key"),"overall","reviewerID","summary", "unixReviewTime", "reviewText")

# change col datatypes
magSub_df = magSub_df.select(col(magSub_df.columns[0]), col(magSub_df.columns[1]).cast('int'), col(magSub_df.columns[2]), 
                             col(magSub_df.columns[3]), col(magSub_df.columns[4]).cast('long'), col(magSub_df.columns[5]))

# replace null values in summary column with empty string
magSub_df = magSub_df.na.fill({'summary': ''})

# Concatenate the two columns and create a new column 'fullText'
magSub_df = magSub_df.withColumn('fullText', concat(col('summary'), col('reviewText')))
magSub_df.show()

+--------------------+-------+--------------+--------------------+--------------+--------------------+--------------------+
|         primary_key|overall|    reviewerID|             summary|unixReviewTime|          reviewText|            fullText|
+--------------------+-------+--------------+--------------------+--------------+--------------------+--------------------+
|AH2IFH762VY5U1005...|      5| AH2IFH762VY5U|AVID READER SINCE...|    1005177600|for computer enth...|AVID READER SINCE...|
|AOSFI0JEYU4XM1004...|      5| AOSFI0JEYU4XM|  The straight scoop|    1004486400|Thank god this is...|The straight scoo...|
|A3JPFWKS83R49V117...|      3|A3JPFWKS83R49V|Antiques Magazine...|    1174694400|Antiques Magazine...|Antiques Magazine...|
|A19FKU6JZQ2ECJ116...|      5|A19FKU6JZQ2ECJ|THE  DISCERNING R...|    1163116800|This beautiful ma...|THE  DISCERNING R...|
|A25MDGOMZ2GALN140...|      5|A25MDGOMZ2GALN|          Five Stars|    1405296000|A great read ever...|Five StarsA great...|
|A3XT9XX

In [ ]:
# Train / Test / Val split
magSub_splits = magSub_df.randomSplit([0.7, 0.1, 0.2], 24)

train_magSub = magSub_splits[0]
val_magSub = magSub_splits[1]
test_magSub = magSub_splits[2]

In [ ]:
train_magSub = train_magSub.withColumn("label", ((train_magSub["overall"] >= 3).cast("int")))
train_magSub_svm = train_magSub.alias('train_magSub_svm')
train_magSub_lr = train_magSub.alias('train_magSub_lr')

val_magSub = val_magSub.withColumn("label", ((val_magSub["overall"] >= 3).cast("int")))
val_magSub_svm = val_magSub.alias('val_magSub_svm')
val_magSub_lr = val_magSub.alias('val_magSub_lr')

test_magSub = test_magSub.withColumn("label", ((test_magSub["overall"] >= 3).cast("int")))
test_magSub_svm = test_magSub.alias('test_magSub_svm')
test_magSub_lr = test_magSub.alias('test_magSub_lr')

In [ ]:
from pyspark.ml.classification import LinearSVC

In [ ]:
# Define the tokenizer, hashingTF, IDF, and ML objects
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="features", labelCol="label") # naive
svm = LinearSVC(maxIter=10, regParam=0.1, labelCol="label", featuresCol="features") # svc
lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=300, regParam=0.001) # lr

# Define the pipeline to execute the stages in order
pipeline_nb = Pipeline(stages=[tokenizer, hashingTF, idf, nb])
pipeline_svm = Pipeline(stages=[tokenizer, hashingTF, idf, svm])
pipeline_lr = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

In [ ]:
import time

In [ ]:
nb_time = time.time()

# Train the Naive Bayes model on the training set
model_nb = pipeline_nb.fit(train_magSub)
predictions = model_nb.transform(val_magSub)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on validation set: {accuracy}')

# Test the model
predictions = model_nb.transform(test_magSub)
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on test set: {accuracy}')

# Precision, Recall, F1-score
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedPrecision')
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedRecall')
recall = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='f1')
f1_score = evaluator.evaluate(predictions)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-Score: {f1_score:.3f}')

print('Time taken for Naive Bayes algorithm:', time.time() - nb_time)

Accuracy on validation set: 0.8505246706854208
Accuracy on test set: 0.8488463261648745
Precision: 0.867
Recall: 0.849
F1-Score: 0.856
Time taken for Naive Bayes algorithm: 33.044373512268066


In [ ]:
svm_time = time.time()

# Train the Support Vector Machine model on the training set
model_svm = pipeline_svm.fit(train_magSub_svm)
predictions = model_svm.transform(val_magSub_svm)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on validation set: {accuracy}')

# Test the model
predictions = model_svm.transform(test_magSub_svm)
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on test set: {accuracy}')

# Precision, Recall, F1-score
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedPrecision')
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedRecall')
recall = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='f1')
f1_score = evaluator.evaluate(predictions)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-Score: {f1_score:.3f}')

print('Time taken for Support Vector Machine algorithm:', time.time() - svm_time)

Accuracy on validation set: 0.8754186202277294
Accuracy on test set: 0.8689516129032258
Precision: 0.860
Recall: 0.869
F1-Score: 0.862
Time taken for Support Vector Machine algorithm: 41.58541393280029


In [ ]:
lr_time = time.time()

# Train the Logistic Regression model on the training set
model_lr = pipeline_lr.fit(train_magSub_lr)
predictions = model_lr.transform(val_magSub_lr)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on validation set: {accuracy}')

# Test the model
predictions = model_lr.transform(test_magSub_lr)
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy on test set: {accuracy}')

# Precision, Recall, F1-score
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedPrecision')
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedRecall')
recall = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='f1')
f1_score = evaluator.evaluate(predictions)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-Score: {f1_score:.3f}')

print('Time taken for Logistic Regression algorithm:', time.time() - lr_time)

Accuracy on validation set: 0.8763116767135521
Accuracy on test set: 0.8774641577060932
Precision: 0.873
Recall: 0.877
F1-Score: 0.875
Time taken for Logistic Regression algorithm: 103.69408512115479


To analyze the latency and throughput of the model, you can use the spark-submit command to run the model on a cluster with a larger dataset. You can monitor the execution time and resource utilization during the execution to identify any bottlenecks or performance issues.

Additionally, you can use the Spark UI to monitor the performance of the model. The Spark UI provides information about the tasks, stages, and resource utilization of the job. You can use this information to optimize the performance of the model.

Another way to analyze the latency and throughput of the model is to use the Spark Streaming module. Spark Streaming can be used to process streaming data in real-time and can help you analyze the performance of the model in a streaming scenario.

Overall, analyzing the latency and throughput of the model can help you optimize the performance and scalability of the model.

After evaluating the model on the validation set, you can perform an error analysis to identify the areas where the model is making errors. This can help you improve the model further by addressing the errors.

One way to perform error analysis is to analyze the confusion matrix. You can use the MulticlassMetrics class from Spark ML to compute the confusion matrix and evaluation metrics such as precision, recall, and F1-score.